## Problem Definition
### In this scenario, Person A wants to open an Italian Restaurant in Toronto. The objective is to solve the problem by using location data from Foursquare and by doing a study of restaurants on all neighborhoods on interest through a combination of location profiling and machine learning.

## Data Understanding
### We are going to explore the neighborhoods of Toronto and their venues in order to gain a better understanding on existing places and to narrow down our neighborhood choices.



In [ ]:
#Install beautifulsoup
!conda install -c conda-forge beautifulsoup4 --yes

# Import packages
from bs4 import BeautifulSoup
import urllib as ur
import requests as rq

In [3]:

# Use Beautiful soup
source = rq.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
article = soup.find('table', class_='wikitable sortable')

In [7]:
# Preprocessing to get the right data frame
codes_list=[]
borough_list=[]
neighborhood_list=[]
i=1
for tag in soup.table.find_all('td'):
    if i == 1:
        codes_list.append(tag.text)
    if i == 2:
        borough_list.append(tag.text)
    if i == 3: 
        row = tag.text
        row = row.replace('\n', '')
        neighborhood_list.append(row)
    i = i+1
    if i==4:
        i=1
        
len(neighborhood_list[0:])

289

In [8]:
# Convert list to pandas dataframe
import pandas as pd
Canada_Codes = pd.DataFrame(
    {'Postcode': codes_list,
     'Borough': borough_list,
     'Neighbourhood': neighborhood_list
    })

In [10]:
Canada_Codes2 = Canada_Codes
Canada_Codes2.drop(Canada_Codes2[Canada_Codes2['Borough']=="Not assigned"].index,axis=0, inplace=True)
Canada_Codes2=Canada_Codes2.groupby("Postcode").agg(lambda x:','.join(set(x)))
Canada_Codes2.loc[Canada_Codes2['Neighbourhood']=="Not assigned",'Neighbourhood']=Canada_Codes2.loc[Canada_Codes2['Neighbourhood']=="Not assigned",'Borough']
Canada_Codes2.index.name = 'Postcode'
Canada_Codes2.reset_index(inplace=True)
Canada_Codes2.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park"
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge"
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [11]:
# Read Geo Data
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")
geo_data.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [12]:
# Create new columns and add it original frame
Canada_Codes2['Latitude']=geo_data['Latitude'].values
Canada_Codes2['Longitude']=geo_data['Longitude'].values

In [13]:
## Final Dataframe with the neighborhood along with latitude and longitude values
Canada_Codes2.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


In [14]:
## Find out how many rows we have.
Canada_Codes2.shape

(103, 5)

In [15]:
import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Importing to use the Foursquare API lab
import folium # map rendering library

print('Foursquare and map plotting Libraries imported.')

Foursquare and map plotting Libraries imported.


In [18]:
print('The dataframe has {} boroughs spanning across {} Postcodes and {}  neighborhood groups.'.format(
        len(Canada_Codes2['Borough'].unique()), len(Canada_Codes2['Postcode'].unique()),
        Canada_Codes2.shape[0]
    )
)

The dataframe has 11 boroughs spanning across 103 Postcodes and 103  neighborhood groups.


In [20]:
import time 
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="capstone_agent")

# Get Toronto geo info
address = 'Toronto, Ontario, Canada'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [38]:
# Get Toronto codes
Toronto_Codes = Canada_Codes2.drop(Canada_Codes2[Canada_Codes2['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
Toronto_Codes.index = pd.RangeIndex(len(Toronto_Codes.index))

#to view Dataframe
Toronto_Codes.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Summerhill East,Moore Park",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West,Forest Hill SE,Deer Park,South...",43.686412,-79.400049


In [41]:
## Get the json results based on Toronot lattude and longitude values
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id=W2RZKFIYGSSLPRR3OE14Q2UYYK5MQZYBV42IYCADG5YZ4PZR&client_secret=2YHEQZDDQ2HBTLH3PFONKEW2DAMNXKSW1HZP5VECDLNFRWWC&ll=43.653963,-79.3872076&v=20180604&radius=500&limit=30'
# get results
results = rq.get(url).json()

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [53]:
# Get nearby venues for Toronto 
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Textile Museum of Canada,Art Museum,43.654396,-79.386500
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
3,Japango,Sushi Restaurant,43.655268,-79.385165
4,Cafe Plenty,Café,43.654571,-79.389450


In [28]:
# Store Credentials
CLIENT_ID = 'W2RZKFIYGSSLPRR3OE14Q2UYYK5MQZYBV42IYCADG5YZ4PZR' # your Foursquare ID
CLIENT_SECRET = '2YHEQZDDQ2HBTLH3PFONKEW2DAMNXKSW1HZP5VECDLNFRWWC' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [66]:
# Get venue and other info for all neighborhoods of Tornoto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = rq.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
toronto_venues = pd.DataFrame(getNearbyVenues(names=Toronto_Codes['Neighbourhood'],
                                   latitudes=Toronto_Codes['Latitude'],
                                   longitudes=Toronto_Codes['Longitude']
                                  ))
print(toronto_venues.shape)


The Beaches
Riverdale,The Danforth West
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East,Moore Park
Summerhill West,Forest Hill SE,Deer Park,South Hill,Rathnelly
Rosedale
St. James Town,Cabbagetown
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,Richmond,King
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill West,Forest Hill North
Yorkville,The Annex,North Midtown
University of Toronto,Harbord
Kensington Market,Grange Park,Chinatown
South Niagara,Bathurst Quay,King and Spadina,Railway Lands,CN Tower,Harbourfront West,Island airport
Stn A PO Boxes 25 The Esplanade
Underground city,First Canadian Place
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Exhibition Place,Parkdale Village,Brockton
High Park,The Junction South
Parkdale,Roncesvall

In [68]:
## View the top 10 to make sure it is working
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"Riverdale,The Danforth West",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,"Riverdale,The Danforth West",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
6,"Riverdale,The Danforth West",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,"Riverdale,The Danforth West",43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant
8,"Riverdale,The Danforth West",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
9,"Riverdale,The Danforth West",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant


In [69]:
# Show the count of venues for each neighborhood. 30 is the maximum limit on number of venues per location request
toronto_venues.groupby('Neighborhood').count().iloc[0:,4]

Neighborhood
Adelaide,Richmond,King                                                                                  30
Berczy Park                                                                                             30
Business Reply Mail Processing Centre 969 Eastern                                                       19
Central Bay Street                                                                                      30
Christie                                                                                                16
Church and Wellesley                                                                                    30
Commerce Court,Victoria Hotel                                                                           30
Davisville                                                                                              30
Davisville North                                                                                         9
Design Exchange,Toronto 

In [75]:
## Explore all the venues returned for the Neighborhood of St.James Town
toronto_venues[toronto_venues.Neighborhood=='St. James Town']


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
289,St. James Town,43.651494,-79.375418,Terroni,43.650927,-79.375602,Italian Restaurant
290,St. James Town,43.651494,-79.375418,Gyu-Kaku Japanese BBQ,43.651422,-79.375047,Japanese Restaurant
291,St. James Town,43.651494,-79.375418,GEORGE Restaurant,43.653346,-79.374445,Restaurant
292,St. James Town,43.651494,-79.375418,Crepe TO,43.650063,-79.374587,Creperie
293,St. James Town,43.651494,-79.375418,Fahrenheit Coffee,43.652384,-79.372719,Coffee Shop
294,St. James Town,43.651494,-79.375418,Triple A Bar (AAA),43.651658,-79.372720,BBQ Joint
295,St. James Town,43.651494,-79.375418,Pearl Diver,43.651481,-79.373600,Gastropub
296,St. James Town,43.651494,-79.375418,Hogtown Smoke,43.649287,-79.374689,Food Truck
297,St. James Town,43.651494,-79.375418,Mystic Muffin,43.652484,-79.372655,Middle Eastern Restaurant
298,St. James Town,43.651494,-79.375418,St James Anglican Cathedral,43.650110,-79.374292,Church


In [ ]:
##Explore the dataframe

print('The dataframe has {} neighborhoods with a total of  {} unique venues spanning across {} venue categories .'.format(
        len(toronto_venues['Neighborhood'].unique()), len(toronto_venues['Venue'].unique()),
        len(toronto_venues['Venue Category'].unique())
    )
)


## Next Steps
### Now we have the data pulled and ready to explore on what would be to explore all the venues in each neighborhood. We can then find the most common venue categories in each neighborhood that are related to restaurants. We can then build a frequency distribution of all restaurant related venue categories for each neighborhood. We can cluster the neighborhoods mean values of frequency for all the relevant restaurant categories.  This will in turn group the neighborhoods based on the presence of certain similar venue categories in one cluster and dissimilar venue categories in another cluster and so on. We can then profile each clusters to understand their behavior. Then depending on the nature of the restaurant type we are interested in opening like Italian etc., we can find the cluster of neighborhoods with least competition for the Italian cuisine but also a hot spot for the restaurants as a whole. 